In [1]:
import tensorflow as tf;import tensorflow.keras as keras;import numpy as np;
from sklearn.datasets import fetch_california_housing;from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split;import os

C:\Users\SONY\miniconda3\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\SONY\miniconda3\lib\site-packages\numpy\.libs\libopenblas.NOIJJG62EMASZI6NYURL6JBKM4EVBGM7.gfortran-win_amd64.dll
C:\Users\SONY\miniconda3\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [2]:
housing = fetch_california_housing()
X_train_full, X_test, y_train_full, y_test = train_test_split(
    housing.data, housing.target.reshape(-1, 1), random_state=42)
X_train, X_valid, y_train, y_valid = train_test_split(
    X_train_full, y_train_full, random_state=42)

scaler = StandardScaler()
scaler.fit(X_train)
X_mean = scaler.mean_
X_std = scaler.scale_

In [3]:
datos_entrenamiento=np.c_[X_train,y_train]
datos_validacion=np.c_[X_valid,y_valid]
datos_test= np.c_[X_test,y_test]

In [58]:
def crear_archivos(data,prefijo,nombres_col=None,particiones=10):
    os.makedirs('data', exist_ok=True)
    m=len(data)
    filepaths = []
    for index,particion_data in enumerate(np.array_split(np.arange(m),particiones)):
        direccion=f'data/my_{prefijo}_{index}.csv'
        filepaths.append(direccion)
        with open(direccion,'wt',encoding='utf-8') as f:
            
            if nombres_col is not None:
                f.write(nombres_col)
                f.write("\n")
            for row_idx in particion_data:
                f.write(",".join([repr(col) for col in data[row_idx]]))
                f.write("\n")
                
    return filepaths
        
    

In [19]:
header_cols = housing.feature_names + ["MedianHouseValue"]
header = ",".join(header_cols)

train_filepaths = crear_archivos(datos_entrenamiento, "train", header, particiones=20)



In [23]:
import pandas as pd

pd.read_csv(train_filepaths[19])

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,MedianHouseValue
0,2.4792,24.0,3.454704,1.134146,2251.0,3.921603,34.18,-118.38,2.000
1,3.9543,35.0,5.134122,0.950690,1305.0,2.573964,33.94,-118.00,2.144
2,3.3750,8.0,5.671233,1.150685,413.0,2.828767,37.10,-119.85,0.807
3,2.9571,28.0,5.215760,0.956848,1638.0,3.073171,38.48,-121.45,1.031
4,3.6786,27.0,5.929078,1.060284,757.0,2.684397,38.64,-121.37,1.597
...,...,...,...,...,...,...,...,...,...
575,2.3375,30.0,3.371681,1.008850,840.0,3.716814,34.29,-118.45,1.542
576,4.0977,50.0,4.788009,1.036403,1171.0,2.507495,37.76,-122.48,3.221
577,1.5561,35.0,4.139932,1.220137,1920.0,3.276451,34.11,-117.29,0.680
578,2.1055,52.0,5.178495,1.070968,1361.0,2.926882,36.76,-119.79,0.613


In [4]:
direcciones=[f'data/my_train_{index}.csv' for index in range(20)] # ya se han creado los archivos.
file_dataset=tf.data.Dataset.list_files(direcciones,seed=42)

for path in file_dataset:
    print(path)


tf.Tensor(b'data\\my_train_13.csv', shape=(), dtype=string)
tf.Tensor(b'data\\my_train_6.csv', shape=(), dtype=string)
tf.Tensor(b'data\\my_train_1.csv', shape=(), dtype=string)
tf.Tensor(b'data\\my_train_7.csv', shape=(), dtype=string)
tf.Tensor(b'data\\my_train_0.csv', shape=(), dtype=string)
tf.Tensor(b'data\\my_train_4.csv', shape=(), dtype=string)
tf.Tensor(b'data\\my_train_18.csv', shape=(), dtype=string)
tf.Tensor(b'data\\my_train_10.csv', shape=(), dtype=string)
tf.Tensor(b'data\\my_train_2.csv', shape=(), dtype=string)
tf.Tensor(b'data\\my_train_9.csv', shape=(), dtype=string)
tf.Tensor(b'data\\my_train_15.csv', shape=(), dtype=string)
tf.Tensor(b'data\\my_train_17.csv', shape=(), dtype=string)
tf.Tensor(b'data\\my_train_3.csv', shape=(), dtype=string)
tf.Tensor(b'data\\my_train_5.csv', shape=(), dtype=string)
tf.Tensor(b'data\\my_train_19.csv', shape=(), dtype=string)
tf.Tensor(b'data\\my_train_8.csv', shape=(), dtype=string)
tf.Tensor(b'data\\my_train_12.csv', shape=(), dtyp

In [5]:
ciclos=5

dataset= file_dataset.interleave(lambda file_path: tf.data.TextLineDataset(file_path).skip(1),
    cycle_length=ciclos)

In [6]:
len(list(dataset.as_numpy_iterator()))

11610

## Preprocessing the Data

(3870, 8)

In [45]:
X_mean,X_std= X_train.mean(axis=0),X_train.std(axis=0)
n_inputs=8

@tf.function
def preprocess(line):
    defs = [0.] * n_inputs + [tf.constant([], dtype=tf.float32)]
    fields = tf.io.decode_csv(line, record_defaults=defs)
    x = tf.stack(fields[:-1])
    y = tf.stack(fields[-1:])
    return (x - X_mean) / X_std, y

In [55]:
def csv_reader_dataset(filepaths, repeat=None, n_readers=5,
n_read_threads=None, shuffle_buffer_size=10000,
n_parse_threads=5, batch_size=32):
    
    dataset = tf.data.Dataset.list_files(filepaths).repeat(repeat)
    dataset = dataset.interleave(
    lambda filepath: tf.data.TextLineDataset(filepath).skip(1),
    cycle_length=n_readers, num_parallel_calls=n_read_threads)
    dataset = dataset.shuffle(shuffle_buffer_size)
    dataset = dataset.map(preprocess, num_parallel_calls=n_parse_threads)
    dataset = dataset.batch(batch_size)
    return dataset.prefetch(tf.data.experimental.AUTOTUNE)



In [56]:
train_set = csv_reader_dataset(direcciones, batch_size=3)
for X_batch, y_batch in train_set.take(2):
    print("X =", X_batch)
    print("y =", y_batch)
    print()

X = tf.Tensor(
[[-0.36341503  1.1372159  -0.65260047 -0.06100351 -0.66828614 -0.4798079
  -1.3616015   1.2116159 ]
 [-0.47644237  0.26702586  0.4395369   0.03383708 -1.1738713  -0.12917034
   1.1119678  -0.8424224 ]
 [-0.30417782 -0.91959685 -0.0151032  -0.16244304  0.24614765 -0.17588145
   1.364947   -0.8724111 ]], shape=(3, 8), dtype=float32)
y = tf.Tensor(
[[4.667]
 [1.55 ]
 [1.094]], shape=(3, 1), dtype=float32)

X = tf.Tensor(
[[ 5.93755484e-01  5.83458602e-01  1.24788165e-01 -2.31382385e-01
  -9.45719182e-01  7.20619857e-02 -8.69698763e-01  6.21894658e-01]
 [ 8.42421204e-02  5.83458602e-01  7.19774514e-02 -7.17237825e-03
   6.55937416e-04  2.37001970e-01 -7.94742167e-01  1.07668054e+00]
 [ 1.22162357e-01  1.08809508e-01 -2.89407790e-01 -6.01498894e-02
  -1.49924487e-01 -4.72927302e-01  1.05106497e+00 -1.26722729e+00]], shape=(3, 8), dtype=float32)
y = tf.Tensor(
[[3.5  ]
 [1.045]
 [2.571]], shape=(3, 1), dtype=float32)



In [60]:
header_cols = housing.feature_names + ["MedianHouseValue"]
header = ",".join(header_cols)

valid_filepaths = crear_archivos(datos_validacion, "validacion", header, particiones=20)

In [61]:
test_filepaths=crear_archivos(datos_validacion, "test", header, particiones=20)

In [63]:
train_set = csv_reader_dataset(direcciones, batch_size=32)
valid_set=csv_reader_dataset(valid_filepaths, batch_size=32)
test_set=csv_reader_dataset(test_filepaths,batch_size=32)

In [73]:
batch_size=32

modelo= keras.models.Sequential([keras.layers.Dense(30,activation='relu',input_shape
                                                =X_train.shape[1:]),keras.layers.Dense(1)])

modelo.compile(loss='mse',optimizer=keras.optimizers.SGD(1e-3))

hist= modelo.fit(train_set,validation_data=valid_set,epochs=10,steps_per_epoch=len(X_train) // batch_size)

Epoch 1/10
341/362 [===========================>..] - ETA: 0s - loss: 2.7099

KeyboardInterrupt: 